# Federated Auto Encoder with Flex for time series Anomaly detection

In [11]:
from flexanomalies.utils import AutoEncoder
from flexanomalies.utils.load_data import split_data, federate_data
from flexanomalies.datasets.preprocessing_utils import (
    create_windows,
    encode_and_bind,
    scaling,
    impute_lost_values,
)
from flexanomalies.utils.metrics import print_metrics
from flexanomalies.utils.process_scores import (
    process_scores_with_percentile,
    process_scores_with_threshold,
)
from sklearn.preprocessing import StandardScaler
from flexanomalies.pool.primitives_deepmodel import (
    build_server_model_ae,
    copy_model_to_clients_ae,
    train_ae,
    set_aggregated_weights_ae,
    weights_collector_ae,
    evaluate_global_model,
    evaluate_global_model_clients,
    threshold_collector_ae,
)
from flexanomalies.pool.aggregators_favg import aggregate_ae
from flexanomalies.pool.aggregators_stats import aggregate_stats_mean
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool
from flexanomalies.utils.metrics import *
import pandas as pd
import numpy as np

Load data, preprocessing and define model parameters

In [12]:
file_path = "../flex-anomalies/flexanomalies/datasets/data/corrected.gz"
split_test = 0.3

df = pd.read_csv(file_path, header=None)

# process labels
df.loc[df[41] != "normal.", 41] = 1
df.loc[df[41] == "normal.", 41] = 0
labels = df[41]
df = df.drop([41], axis=1)

features_to_encode = [1, 2, 3]
df = df.drop(features_to_encode, axis=1)


In [13]:
model_params = {
    "epochs": 50,
    "input_dim": df.shape[1],
    "batch_size": 32,
    "neurons": [16, 8, 16],
    "hidden_act": ["relu", "relu", "relu"],
    "preprocess":False,
    "w_size": 30,
    "n_pred": 10,
    "contamination": 0.1,
}

Data preprocessing (time series) and definition of the sliding window 

In [14]:
X = scaling(np.array(df.iloc[:, :].astype(float)))
y = np.array(labels)
X_train, X_test, l_train, l_test = split_data(X, y, split_size=0.30)

In [15]:
def create_windows(w_size, n_pred, X_train, X_test, l_train, l_test):
    X_train_windows = []
    y_train_windows = []
    X_test_windows = []
    y_test_windows = []

    for i in range(0, len(X_train), n_pred):
        temp_xtrain = X_train[i : w_size + i, :]
        temp_ytrain = l_train[i : w_size + i]
        if len(temp_xtrain) < w_size or len(temp_ytrain) < n_pred:
            break
        X_train_windows.append(temp_xtrain)

        y_train_windows.append(temp_ytrain)

    for i in range(0, len(X_test), n_pred):

        temp_xtest = X_test[i : w_size + i, :]
        temp_ytest = l_test[i : w_size + i]
        if len(temp_xtest) < w_size or len(temp_ytest) < n_pred:
            break

        X_test_windows.append(temp_xtest)

        y_test_windows.append(temp_ytest)

    return (
        np.array(X_train_windows),
        np.array(y_train_windows),
        np.array(X_test_windows),
        np.array(y_test_windows),
    )

In [ ]:
(
    X_train_windows,
    y_train_windows,
    X_test_windows,
    y_test_windows,
    
) = create_windows(model_params["w_size"], model_params["n_pred"], X_train, X_test,l_train, l_test)

print("X_train shape == {}.".format(np.array(X_train_windows).shape))
print("y_train shape == {}.".format(np.array(y_train_windows).shape))
print("X_test shape == {}.".format(np.array(X_test_windows).shape))
print("y_test shape == {}.".format(np.array(y_test_windows).shape))




In [17]:
model = AutoEncoder(**model_params)


## Creating the federated architecture

Once the data  is loaded and preprocessed, we have to federate it. For this we use the FLEX library. There are two ways to federate the data, using an IID distribution or a non IID distribution. For the IID distribution we can use the ìid_distribution 

function of FedDataDistribution. If we use a non-IID distribution, it is necessary to use a custom configuration, such as the one used in the federate_data function. For more information, go to the FLEX library workbooks, and take a look at the 

Federate Data with FLEXible notebooks.

In [18]:
flex_dataset = federate_data(5, X_train_windows,X_train_windows)
pool = FlexPool.client_server_pool(
    fed_dataset=flex_dataset,
    server_id="autoencoder_server",
    init_func=build_server_model_ae,
    model=model,
)


## Run the federated learning experiment and Evaluate
Now, we can run the federated experiment for multiple rounds using the decorators. 

Once the model is trained, we need to evaluate it at the server level and at client level.

In [ ]:
for i in range(3):
    print(f"\nRunning round: {i}\n")
    pool.servers.map(copy_model_to_clients_ae, pool.clients)
    pool.clients.map(train_ae)
    pool.aggregators.map(weights_collector_ae, pool.clients)
    pool.aggregators.map(aggregate_ae)
    pool.aggregators.map(set_aggregated_weights_ae, pool.servers)
output_model = pool.servers._models["autoencoder_server"]["model"]


In [ ]:
evaluate_global_model(output_model, X_test_windows, X_test_windows, y_test_windows.flatten().astype("int"))

### Evaluating at client level 

To evaluate it at the client level, the function evaluate_global_model_clients is used, which uses the test set in each of the clients, predicts the corresponding values, evaluates and an anomaly threshold is obtained.

The threshold_collector_ae function obtains the threshold for each of the clients and the aggregate_stats_mean function performs the aggregation of the thresholds.

In [ ]:

pool.clients.map(evaluate_global_model_clients)
thresholds = pool.clients.map(threshold_collector_ae)
aggregate_stats_mean(thresholds)

In the end, the threshold is used to evaluate the model at the server level and perform labeling

In [ ]:
evaluate_global_model(
    output_model,
    X_test_windows,
    X_test_windows,
    y_test_windows.flatten().astype("int"),
    threshold=aggregate_stats_mean(thresholds),
)

## End of Notebook